In [1]:
!which python

/Users/tareksanger/Developer/Lumis/.venv/bin/python


In [3]:
page = wikipedia.page(title="Stantec", auto_suggest=True)
page.content

'Stantec Inc. is an international professional services company in the design and consulting industry. The company was founded in 1954, as D. R. Stanley Associates in Edmonton, Alberta, Canada. Stantec provides professional consulting services in planning, engineering, architecture, interior design, landscape architecture, surveying, environmental sciences, project management, and project economics for infrastructure and facilities projects. The company provides services on projects around the world, with over 30,000 employees operating out of more than 450 locations in North America and across offices on six continents.\n\n\n== History ==\nDon Stanley was the first Canadian to earn a Ph.D. in environmental engineering. Attending Harvard University on a Rockefeller Foundation scholarship, he earned his doctorate in 1952 and two years later founded D.R. Stanley & Associates, working as the sole proprietor out of a 250-square-foot (23 m2) office in Edmonton, Alberta. In 1955, Stanley hir

In [4]:
import spacy
from spacy.matcher import Matcher 
from spacy.tokens import Span 

# Load English language model
nlp = spacy.load("en_core_web_sm")

text = "Apple Inc. was founded by Steve in Cupertino, California."
doc = nlp(text)

for token in doc:
    if token.dep_ == "nsubj" and token.head.dep_ == "ROOT":
        print(f"Subject: {token.text}, Action: {token.head.text}")

# Function to extract relationships
def get_entities(sent):
  ## chunk 1
  ent1 = ""
  ent2 = ""

  prv_tok_dep = ""    # dependency tag of previous token in the sentence
  prv_tok_text = ""   # previous token in the sentence

  prefix = ""
  modifier = ""

  #############################################################
  
  for tok in nlp(sent):
    ## chunk 2
    # if token is a punctuation mark then move on to the next token
    if tok.dep_ != "punct":
      # check: token is a compound word or not
      if tok.dep_ == "compound":
        prefix = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          prefix = prv_tok_text + " "+ tok.text
      
      # check: token is a modifier or not
      if tok.dep_.endswith("mod") == True:
        modifier = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          modifier = prv_tok_text + " "+ tok.text
      
      ## chunk 3
      if tok.dep_.find("subj") == True:
        ent1 = modifier +" "+ prefix + " "+ tok.text
        prefix = ""
        modifier = ""
        prv_tok_dep = ""
        prv_tok_text = ""      

      ## chunk 4
      if tok.dep_.find("obj") == True:
        ent2 = modifier +" "+ prefix +" "+ tok.text
        
      ## chunk 5  
      # update variables
      prv_tok_dep = tok.dep_
      prv_tok_text = tok.text
  #############################################################

  return [ent1.strip(), ent2.strip()]
# Extract and print relationships
relationships = get_entities(text)
for rel in relationships:
    print(rel)

Apple Inc.
Cupertino


In [5]:
def get_relation(sent):

  doc = nlp(sent)

  # Matcher class object 
  matcher = Matcher(nlp.vocab)

  #define the pattern 
  pattern = [{'DEP':'ROOT'}, 
            {'DEP':'prep','OP':"?"},
            {'DEP':'agent','OP':"?"},  
            {'POS':'ADJ','OP':"?"}] 

  matcher.add("matching_1",[pattern]) 

  matches = matcher(doc)
  k = len(matches) - 1

  span = doc[matches[k][1]:matches[k][2]] 

  return(span.text)

In [6]:
get_relation(text)

'founded by'

In [7]:
# from transformers import pipeline
# qa = pipeline("question-answering")
# result = qa(question="Who founded Apple?", context="Apple was founded by Steve Jobs in 1976.")
# print(result['answer'])

import spacy
nlp = spacy.load("en_core_web_sm")
text = "Apple was founded by Steve Jobs in 1976 ."
doc = nlp(text)
for ent in doc.ents:
    print(ent.text, ent.label_)

Apple ORG
Steve Jobs PERSON
1976 DATE
